## Overview
---

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/aws_rekognition/find_face_and_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

👨‍🍳 This section of our cookbook demonstrates a method for using video analysis to identify and highlight significant moments involving people in a video

🥡 Key components of this technique include::
- **AWS Rekognition API**: The [FaceSearch](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceSearch.html) endpoint of the AWS Rekognition API will be utilized to scan the video and detect the presence of individuals.
- **VideoDB**: This tool will be used for storing the video in a database specifically designed for videos. It also aids in extracting and compiling clips of the identified person into a master clip.

🌶️ Most spicy thing about our dish ?   

We will collect timestamps of persons presence, get clips from video, merge them all,  **without needing to touch video editor, waiting in render queue and instantly playable**

## Setup
---

### 🔧 Installing Required Packages

To ensure our Python environment has the necessary tools, we need to install following packages:
- boto3: to use aws services such as [S3](https://docs.aws.amazon.com/s3/) and [AWS rekognition api](https://docs.aws.amazon.com/rekognition/)
- pytube: for downloading YouTube Videos.
- videodb : to access videodb

In [ ]:
!pip install boto3 pytube requests videodb

### Helper functions
- `download_video_yt` : downloads a YouTube video at highest resolution, saving it with a specified filename.
- `download_file` : downloads any file from a URL using wget, saving it with a specified name.

In [3]:
import requests
import pytube
import os
import datetime
import time
import json

#Downloads Youtube video
def download_video_yt(url, output_file="video.mp4"):
    youtube_object = pytube.YouTube(url)
    video_stream = youtube_object.streams.get_highest_resolution()
    video_stream.download(filename=output_file)
    print(f"Downloaded {url} as {output_file}")
    return output_file


#Downloads Image
def download_file(url, output_file="image.jpg"):
    command = f"wget -O {output_file} {url}"
    exit_status = os.system(command)
    if exit_status == 0:
        print(f"Downloaded {url} as {output_file}")
    else:
        print("An error occurred")


### Downloading media
Downloading and processing media. 

We will download a YouTube video and extract clips featuring a specific individual to create a new video focused on that person.   
This demo will use a 15-minute Silicon Valley scene featuring Martin Starr(Gilfoyle), with a specific clip extracted for demonstration purposes

But, you can change target media as per your needs using `video_url_yt_sv` and `target_person` 


<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
    <strong>Attention:</strong> We are using a paid API, so please select your YouTube video carefully. Choosing a larger video may incur additional charges.
</div>

In [42]:
video_url_yt_sv = "https://www.youtube.com/watch?v=NNAgJ5p4CIY"
image_gilfoyle = "https://static.hbo.com/content/dam/hbodata/series/silicon-valley/characters/s3/gilfoyle-1920.jpg"
image_jinyang = "https://static.hbo.com/content/dam/hbodata/series/silicon-valley/characters/s2/jian-yang-1920.jpg"
image_erlic = "https://static.wikia.nocookie.net/silicon-valley/images/1/1f/Erlich_Bachman.jpg"
image_jared = "https://static.hbo.com/content/dam/hbodata/series/silicon-valley/characters/s3/jared-s3-1920.jpg"
image_dinesh = "https://static.wikia.nocookie.net/silicon-valley/images/e/e3/Dinesh_Chugtai.jpg"
image_richard = "https://static.wikia.nocookie.net/silicon-valley/images/3/33/Richard_Hendricks.jpg"

indexed_faces = [{
    "img": image_gilfoyle,
    "output": "image_gilfoyle.jpg",
    "userid": "gilfoyle1"
},
              {
    "img": image_jinyang,
    "output": "image_jinyang.jpg",
    "userid": "jinyang"
},
              {
    "img": image_erlic,
    "output": "image_erlic.jpg",
    "userid": "erlic"
},
                  {
    "img": image_jared,
    "output": "image_jared.jpg",
    "userid": "jared"
},
                {
    "img": image_dinesh,
    "output": "image_dinesh.jpg",
    "userid": "dinesh"
},
                {
    "img": image_richard,
    "output": "image_richard.jpg",
    "userid": "richard"
}
               ]

video_output = "video.mp4"
image_output = "image.jpg"

download_video_yt(video_url_yt_sv, video_output)

for indexed_face in indexed_faces:
  download_file(indexed_face['img'],indexed_face['output'] )


Downloaded video to: video.mp4
Downloaded image to: image.jpg


'image.jpg'

## ⚙️ Configuartion
We must set up AWS and the VideoDB api keys.

### 🔗 Setting Up a connection to db

To connect to `VideoDB`, simply create a `Connection` object.

This can be done by either providing your VideoDB API key directly to the constructor or by setting the `VIDEO_DB_API_KEY` environment variable with your API key.

> 💡 Your API key is available in the [VideoDB dashboard](https://console.videodb.io)

In [ ]:
from videodb import connect, play_stream
conn = connect(api_key="")

### 🛠️ AWS Configuration

- AWS secrets like `aws_secret_key_id`, `aws_secret_access_key` and `aws_reigon` 
- `collection_id` - This refers to the AWS collection in which we will store the indexed faces.  
- `bucket_name` - This is the name of the S3 bucket where the video will be stored.  


In [ ]:
import boto3

aws_access_key_id= "YOUR_AWS_KEY_ID"
aws_secret_access_key = "YOUR_AWS_SECRET_KEY" 
region_name = "YOUR_AWS_REIGON"

bucket_name = "videorekog"
collection_id = "videorekog"


rekognition_client = boto3.client(
    "rekognition",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
)
s3 = boto3.client('s3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
)

## Rekognition API Workflow

- Create a collection for faces to store
- Create a new user with given `user_id`
- Index Face from image and store index in `collection_id`
- Associate Index face with newly created user
- Upload a video to S3 Bucket and Start face search using [StartFaceSearch](https://docs.aws.amazon.com/rekognition/latest/dg/search-face-with-image-procedure.html)

In [ ]:
def index_faces_from_image(image_path, collection_id):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()

    response = rekognition_client.index_faces(
        CollectionId=collection_id,
        DetectionAttributes=["ALL"],
        Image={"Bytes": image_bytes},
    )

    if not response["FaceRecords"]:
        print(f"No faces detected in image: {image_path}")
    return response["FaceRecords"][0]["Face"]["FaceId"]



for indexed_face in indexed_faces:
  rekognition_client.create_user(UserId=indexed_face["userid"], CollectionId=collection_id)
  face_id = index_faces_from_image(indexed_face["output"], collection_id)
  rekognition_client.associate_faces(CollectionId=collection_id, UserId=indexed_face["userid"], FaceIds=[face_id])
  indexed_face["faceid"] = face_id

In [ ]:
# Start a Face Search Job 
def start_face_search(video_path, collection_id, bucket_name):
    response = rekognition_client.start_face_search(
        Video={"S3Object": {"Bucket": bucket_name, "Name": video_path}},
        CollectionId=collection_id
    )

    return response["JobId"]

# Get Result of Face Search Job
def get_face_search_results(job_id):
    wait_for = 5
    while True:
        response = rekognition_client.get_face_search(JobId=job_id)
        status = response["JobStatus"]
        if status == "IN_PROGRESS":
            time.sleep(wait_for)
        else:
            break

    if status == "SUCCEEDED":
        return response
    else:
        print(f"Face search failed with status: {status}")
        return None

# Upload our video to S3 Bucket
s3.create_bucket(Bucket=bucket_name)
s3.upload_file(video_output, bucket_name, video_output)

# Search faces in uploaded video using Rekogntion API 
job_id = start_face_search(video_output, collection_id, bucket_name )
print(job_id)
face_res = get_face_search_results(job_id)

## Extracting Shots from Video

### Preparing Clips Timestamps
When the Rekognition API detects indexed faces from a collection, it provides timestamps. Our goal is to merge timestamps that are part of the same scene.  
While the [AWS Segment API](https://docs.aws.amazon.com/rekognition/latest/dg/segment-api.html) is one way to do this, we'll use a simpler approach.
If the interval between two timestamps is smaller than a specified `threshold`, they will be grouped as a single shot. 
Additionally, we will apply a `padding` on both the right and left sides of each shot for better context.

You can experiment with both variables `threshold` and `padding` to get a better result.

In [ ]:
threshold = 3
padding = 3

for indexed_face in indexed_faces:
   indexed_face["timestamps"] = []

for person in face_res["Persons"]:
    if "FaceMatches" in person and person["FaceMatches"]:
      for face in person["FaceMatches"]:
        face_id = face["Face"]["FaceId"]
        for indexed_face in indexed_faces:
          if indexed_face["faceid"] == face_id:
            seconds = person["Timestamp"]/1000
            indexed_face["timestamps"].append(seconds)


def merge_timestamps(numbers, threshold, padding):
    grouped_numbers = [[0, 0]]
    current_group = [numbers[0]]

    for i in range(1, len(numbers)):
        if numbers[i] - numbers[i-1] <= threshold:
            current_group.append(round(numbers[i]))
        else:
            left_end = current_group[0]-padding
            right_end = current_group[-1]+padding
            if(left_end <0):
              left_end = 0
            if(left_end < grouped_numbers[-1][-1]):
              grouped_numbers[-1][-1] = right_end
            else:
              grouped_numbers.append([left_end, right_end])
            current_group = [round(numbers[i])]

    grouped_numbers.append([current_group[0] - padding, current_group[-1]+padding])
    return grouped_numbers

for indexed_face in indexed_faces:
  indexed_face["shots"] = merge_timestamps(indexed_face["timestamps"], threshold, padding )


### Extracting Clips from Video Using videodb

The idea behind videodb is straightforward: it functions as a database specifically for videos. Similar to how you upload tables or JSON data to a standard database, you can upload your videos to videodb. You can also retrieve your videos through queries, much like accessing regular data from a database.

Additionally, videodb enables you to swiftly create clips from your videos, ensuring a ⚡️ process, just like retreiving text data from a db.

For this demo, we'll be uploading our Silicon Valley clip to `videodb`, which has been processed using the AWS Rekognition API, identified by the URL `video_url_yt_sv`. 

Following this, we will compile a master clip composed of smaller segments that depict the individual we have indexed using Rekognition API 


In [ ]:
video = conn.upload(url=video_url_yt_sv)
for indexed_face in indexed_faces:
  indexed_face["stream"] = video.generate_stream(timeline=indexed_face["timeline"])  


## Results 

![Curtains](https://c.tenor.com/3-rQGspkeboAAAAd/tenor.gif)

Now, its time to view our results.


In [ ]:
for indexed_face in indexed_faces:
    if(indexed_face['userid']=="erlic"):
        play_stream(indexed_face["stream"])
